In [1]:
# Setup

# img_directory = '/Users/rwilliams/Desktop/celeba/training'
img_directory = '/home/ec2-user/training-data/img_align_celeba'
# model_save_path = '/home/ec2-user/tf-checkpoints/vae-celeba/checkpoint.ckpt'
outputs_directory = '/home/ec2-user/outputs/vaegan-celeba'
log_directory = '/home/ec2-user/tf-logs/vaegan-celeba'
batch_size = 64
training_set_size = 256
img_size = 128

# for adam optimizer
learning_rate = 2e-4
learning_beta1 = 0.5
# learning_beta1 = 0.9

zsize = 128

# weights similarity loss term for decoder loss
loss_gamma = 1e-1

In [2]:
# Jupyter imports

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import numpy as np
import scipy as sp
import os
from utils import imshow, resize_crop, load_img

In [4]:
# load training data
training = np.array([resize_crop(load_img(i+1, img_directory), (img_size, img_size)) for i in range(training_set_size)])

/home/ec2-user/.conda/envs/keras/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [5]:
# create models

import tensorflow as tf
from discriminator import Discriminator
tf.reset_default_graph()
tf.set_random_seed(42.0)

# input images feed
X1 = tf.placeholder(tf.float32, [None, img_size, img_size, 3])

# for feeding random draws of z (latent variable)
X2 = tf.placeholder(tf.float32, [None, img_size, img_size, 3])

# discriminator attached to X1 input
# shares weights with other discriminator
disc_x1_obj = Discriminator(img_shape=(img_size, img_size, 3))
disc_x1_obj.disc(X1)
disc_x1_logits = disc_x1_obj.logits

disc_x2_obj = Discriminator(img_shape=(img_size, img_size, 3))
disc_x2_obj.disc(X2, reuse=True)
disc_x2_logits = disc_x2_obj.logits

In [6]:
# set up loss functions and training_ops

disc_x1_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    labels=tf.zeros_like(disc_x1_logits),
    logits=disc_x1_logits))

disc_x2_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    labels=tf.ones_like(disc_x2_logits),
    logits=disc_x2_logits))

# minimize these with optimizer
disc_loss = disc_x1_loss + disc_x2_loss

# get weights to train for each of encoder, decoder, etc.
# pass this to optimizer so it only trains w.r.t the network
# we want to train and just uses other parts of the network as is
# (for example use the discriminator to compute a loss during training
# of the encoder, but don't adjust weights of the discriminator)

# need to explicitly add some of the BN variables so they get trained.
disc_update_ops = [i for i in tf.get_collection(tf.GraphKeys.UPDATE_OPS) if 'discriminator' in i.name]
disc_vars = [i for i in tf.trainable_variables() if 'discriminator' in i.name]

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=learning_beta1)

with tf.control_dependencies(disc_update_ops):
    train_disc = optimizer.minimize(disc_loss)

# Init session

In [7]:
# create or restore session

sess = tf.InteractiveSession()
try:
    print('trying to restore session')
    saver.restore(sess, model_save_path)
    print('restored session')
except:
    print('failed to restore session, creating a new one')
    tf.global_variables_initializer().run()

# write logs for tensorboard
writer = tf.summary.FileWriter(log_directory, sess.graph)

trying to restore session
failed to restore session, creating a new one


In [8]:
# collect data for tensorboard

disc_x1_out = tf.reduce_mean(tf.sigmoid(disc_x1_logits))
disc_x2_out = tf.reduce_mean(tf.sigmoid(disc_x2_logits))

tf.summary.scalar('disc_x1_loss', disc_x1_loss)
tf.summary.scalar('disc_x2_loss', disc_x2_loss)
tf.summary.scalar('disc_x1_out', disc_x1_out)
tf.summary.scalar('disc_x2_out', disc_x2_out)
tf.summary.scalar('disc_loss', disc_loss)

merged_summary = tf.summary.merge_all()

# Train

In [9]:
import math
epochs = 10000
batches = int(float(training_set_size) / batch_size)

for epoch in range(epochs):
    print ('epoch %s ' % epoch, end='')
    randos = np.random.normal(size=(training_set_size, img_size, img_size, 3))
    
    # train discriminator
    for batch in range(batches):
        x1feed = training[batch*batch_size:(batch+1)*batch_size]
        x2feed = randos[batch*batch_size:(batch+1)*batch_size]
        sess.run(train_disc, feed_dict={X1: x1feed, X2: x2feed})
        print('.', end='')
               
    print('')
    
    if (epoch % 2 == 0):        
        x1feed = training[:batch_size]
        x2feed = randos[:batch_size]
        summary = merged_summary.eval(feed_dict={X1: x1feed, X2: x2feed})
        writer.add_summary(summary, epoch) 

       

epoch 0 ....
epoch 1 ....
epoch 2 ....
epoch 3 ....
epoch 4 ....
epoch 5 ....
epoch 6 ....
epoch 7 ....
epoch 8 ....
epoch 9 ....
epoch 10 ....
epoch 11 ....
epoch 12 ....
epoch 13 ....
epoch 14 ....
epoch 15 ....
epoch 16 ....
epoch 17 ....
epoch 18 ....
epoch 19 ....
epoch 20 ....
epoch 21 ....
epoch 22 ....
epoch 23 ....
epoch 24 ....
epoch 25 ....
epoch 26 ....
epoch 27 ....
epoch 28 ....
epoch 29 ....
epoch 30 ....
epoch 31 ....
epoch 32 ....
epoch 33 ....
epoch 34 ....
epoch 35 ....
epoch 36 ....
epoch 37 ....
epoch 38 ....
epoch 39 ....
epoch 40 ....
epoch 41 ....
epoch 42 ....
epoch 43 ....
epoch 44 ....
epoch 45 ....
epoch 46 ....
epoch 47 ....
epoch 48 ....
epoch 49 ....
epoch 50 ....
epoch 51 ....
epoch 52 ....
epoch 53 ....
epoch 54 ....
epoch 55 ....
epoch 56 ....
epoch 57 ....
epoch 58 ....
epoch 59 ....
epoch 60 ....
epoch 61 ....
epoch 62 ....
epoch 63 ....
epoch 64 ....
epoch 65 ....
epoch 66 ....
epoch 67 ....
epoch 68 ....
epoch 69 ....
epoch 70 ....
epoch 71 ....
ep

KeyboardInterrupt: 

In [ ]:
# vae_out = decoder.eval(feed_dict={X: training[:4]})
# imshow(training[:4])
# imshow(vae_out[:4])

In [ ]:
# r = np.random.normal(size=(8,128), scale=1.0)
# y = sess.run(decoder, feed_dict={encoder: r})
# imshow(y[0:8])